In [2]:
from langchain_community.document_loaders import TextLoader

loader=TextLoader("html/animation_system.html",encoding="utf-8")
doc=loader.load()
doc

[Document(metadata={'source': 'html/animation_system.html'}, page_content='<!DOCTYPE html>\n<html lang="zh-CN">\n<head>\n    <meta charset="UTF-8">\n    <title>动画系统原理介绍</title>\n    <style>\n        :root {\n            --primary: #3498db;\n            --bg: #f4f7f6;\n            --text: #2c3e50;\n        }\n        body {\n            font-family: \'Segoe UI\', Tahoma, Geneva, Verdana, sans-serif;\n            background-color: var(--bg);\n            color: var(--text);\n            line-height: 1.6;\n            padding: 40px;\n        }\n        .card {\n            background: white;\n            max-width: 900px;\n            margin: 0 auto;\n            padding: 30px;\n            border-radius: 12px;\n            box-shadow: 0 4px 20px rgba(0,0,0,0.08);\n        }\n        h1 { border-left: 5px solid var(--primary); padding-left: 15px; }\n        \n        /* 核心动画演示区 */\n        .demo-box {\n            height: 200px;\n            background: #eee;\n            margin: 20px 0;\

In [3]:
headers_to_spilt_on=[
    ("h1","Header 1"),
    ("h2","Header 2"),
    ("h3","Header 2"),
]

In [6]:
from langchain_text_splitters import HTMLHeaderTextSplitter
html_splitts=HTMLHeaderTextSplitter(headers_to_split_on=headers_to_spilt_on)
html_header_spilts=html_splitts.split_text(doc[0].page_content)
html_header_spilts

[Document(metadata={}, page_content='Web 动画系统概览 技术小贴士'),
 Document(metadata={'Header 1': 'Web 动画系统概览'}, page_content='动画系统是 UI 交互的灵魂。在 Web 开发中，它主要由三个核心部分组成：  \n1. 关键帧 (Keyframes)'),
 Document(metadata={'Header 1': 'Web 动画系统概览', 'Header 2': '1. 关键帧 (Keyframes)'}, page_content='定义动画在特定时间点的状态。例如 0% 是开始，100% 是结束。'),
 Document(metadata={'Header 1': 'Web 动画系统概览'}, page_content='2. 补间/缓动 (Easing)'),
 Document(metadata={'Header 1': 'Web 动画系统概览', 'Header 2': '2. 补间/缓动 (Easing)'}, page_content='决定状态变化的“节奏”。是匀速运动，还是像弹簧一样先快后慢？'),
 Document(metadata={'Header 1': 'Web 动画系统概览'}, page_content='3. 属性转换 (Transforms)'),
 Document(metadata={'Header 1': 'Web 动画系统概览', 'Header 2': '3. 属性转换 (Transforms)'}, page_content='操作元素的位移、旋转、缩放。通过 GPU 加速，确保动画在高负载下依然流畅。'),
 Document(metadata={'Header 1': 'Web 动画系统概览'}, page_content='4. 触发机制 (Triggers)'),
 Document(metadata={'Header 1': 'Web 动画系统概览', 'Header 2': '4. 触发机制 (Triggers)'}, page_content='动画何时开始？可以是页面加载自启动，也可以是用户点击或悬停时触发。'),
 Document(metadata={'Header 1': 'Web 

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(
  base_url="http://127.0.0.1:1234/v1",
  openai_api_key="lm-studio",
  model="text-embedding-bge-large-zh-v1.5",
  check_embedding_ctx_length=False
)

In [7]:
vectorstoreDB=FAISS.from_documents(html_header_spilts,embeddings)
vectorstoreDB

In [8]:
from langchain_core.prompts import ChatPromptTemplate

template="""
只根据以下文档回答问题：
{context}

问题：{question}
"""

prompt=ChatPromptTemplate.from_template(template)

In [9]:
retriver=vectorstoreDB.as_retriever(
     search_type="mmr",
     search_kwargs={"k": 2}
)

In [10]:
docs=retriver.get_relevant_documents("动画是什么")
docs

C:\Users\86151\AppData\Local\Temp\ipykernel_108296\3943019962.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs=retriver.get_relevant_documents("动画是什么")


[Document(metadata={}, page_content='Web 动画系统概览 技术小贴士'),
 Document(metadata={'Header 1': 'Web 动画系统概览', 'Header 2': '1. 关键帧 (Keyframes)'}, page_content='定义动画在特定时间点的状态。例如 0% 是开始，100% 是结束。')]

In [11]:
from langchain_openai import ChatOpenAI,OpenAI

model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.3
)

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

chain=(
    {"context": retriver, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

for chunk in chain.stream("动画是什么"):
    print(chunk,end="")

动画是 Web 动画系统概览 技术小贴士 中提到的，在特定时间点的状态。